In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
Base.prepare(autoload_with=engine)

# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

# Exploratory Precipitation Analysis

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
# Find the most recent date in the data set.
most_recent_date = session.query(func.max(Measurement.date)).scalar()
most_recent_date

#### Design a query to retrieve the last 12 months of precipitation data and plot the results,</br> starting from the most recent data point in the database. 

In [ ]:
# Calculate the date one year previous
one_year_previous = dt.date.fromisoformat(most_recent_date) \
                    - dt.timedelta(days=365)

# Query data from Measurement table 
precip_year = session.query(Measurement.date, func.max(Measurement.prcp))\
    .filter(Measurement.date > one_year_previous)\
    .group_by(Measurement.date)\
    .order_by(Measurement.date)\
    .all()


In [ ]:
# Extracting x and y values from the data
x_values = [item[0] for item in precip_year]
y_values = [item[1] for item in precip_year]

# Set up the plot
plt.figure(figsize=(10, 6))
plt.bar(x_values, y_values)
plt.xlabel('Date')
plt.ylabel('Precipitation (inches)')
plt.title('Precipitation (1 Year)')

# Adjust x-axis tick labels
start_date = one_year_previous
tick_interval = 77  

# Filter x_values and corresponding labels
filtered_x_values = [x for i, x in enumerate(x_values) if i % tick_interval == 0]
filtered_labels = [x for i, x in enumerate(x_values) if i % tick_interval == 0]

plt.xticks(filtered_x_values, filtered_labels, rotation=90)
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data
prcp_summary = pd.DataFrame(session.query(Measurement.date, Measurement.prcp)\
                .filter(Measurement.date >= one_year_previous))
prcp_summary.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number of stations in the dataset
print('The number of Stations is: ', session.query(Station.station).count())

In [ ]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.

station_counts = session.query(Measurement.station, func.count(Measurement.id).label('ct'))\
                        .group_by(Measurement.station)\
                        .order_by(func.count(Measurement.id).desc())\
                        .order_by('ct')\
                        .all()
station_counts

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.

most_active = station_counts.nlargest(1).index[0]

most_active_df = M_data[M_data['station'] == most_active]

mas_lo = most_active_df['tobs'].min()
mas_hi = most_active_df['tobs'].max()
mas_avg = most_active_df['tobs'].mean()

print(f'Lowest Temp: {mas_lo}')
print(f'Highest Temp: {mas_hi}')
print(f'Average Temp: {mas_avg}')

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram

mas_temp_1yr = year_precip[year_precip['station'] == most_active]

# Set up the plot

plt.figure(figsize=(10, 6))  # Set the figure size

# Create a histogram of temperatures with 12 bins
plt.hist(mas_temp_1yr['tobs'], bins=12, color='blue', alpha=0.7)

# Configure the plot
plt.xlabel('Temperature')
plt.ylabel('Frequency')
plt.title('Temperature Observations, past year')
plt.legend(['Temperature'], loc='upper right')

# Show the plot
plt.tight_layout()  
plt.show()



### Close Session

In [ ]:
session.close()